In [29]:
# Code to call IODA API for checking down time of an ASN
import requests
import json
import datetime


start_time = "1735689967"
end_time = "1737835118"
start_time_date = datetime.datetime.fromtimestamp(int(start_time), datetime.timezone.utc)
end_time_date = datetime.datetime.fromtimestamp(int(end_time), datetime.timezone.utc)


asn = "37081"
api_url = "https://api.ioda.inetintel.cc.gatech.edu/v2/outages/events?entityType=asn&entityCode="+asn+"&from="+start_time+"&until="+end_time

response = requests.get(api_url)
# response = requests.get(api_url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert response JSON to a string
    json_string = response.json()
    print(f"Number of times the AS {asn} is down is {len(json_string['data'])} from {start_time_date} to {end_time_date}")
else:
    print(f"Error: Received status code {response.status_code}")   

Number of times the AS 37081 is down is 92 from 2025-01-01 00:06:07+00:00 to 2025-01-25 19:58:38+00:00


In [ ]:
# Create a json like this 
# E.g. {"19574":[{"total_down_times":0},[]],"19905":[{"total_down_times":2},[{"start":1736205900,"duration":1800},{"start":1736654700,"duration":9000}]]}
import json
import datetime

# Assume the json file has all ASNs registered in a country.
def get_unique_asns(country):
    # Open and read the JSON file
    with open('../data/'+country+'_asn.json', 'r') as file:
        data = json.load(file)
    total_asns_nl = [asn for asn_list in data.values() for asn in asn_list]
    unique_asns = list(set(total_asns_nl))
    return unique_asns


countries = ["netherlands", "sweden"]

for country in countries:
    
    # Check downtime of asns of unique_asns of estonia for January 2025
    start_time = "1730415600" # Nov 01 00:00:00 UTC
    end_time = "1733007599" # Nov 30 23:59:59 UTC

    # Change time to UTC
    start_time_date = datetime.datetime.fromtimestamp(int(start_time), datetime.timezone.utc)
    end_time_date = datetime.datetime.fromtimestamp(int(end_time), datetime.timezone.utc)

    down_time_asn = { }

    unique_asns = get_unique_asns(country)
    
    #Iterate over asns 
    for asn in unique_asns:
        api_url = "https://api.ioda.inetintel.cc.gatech.edu/v2/outages/events?entityType=asn&entityCode="+asn+"&from="+start_time+"&until="+end_time

        response = requests.get(api_url)
        # response = requests.get(api_url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Convert response JSON to a string
            json_string = response.json()
            data = json_string['data']


            down_time_counter = len(data)

            # Merge all dictionaries
            merge_all = []

            down_time_counter_dict = { }
            down_time_counter_dict["total_down_times"] = down_time_counter # {'total_down_times': 8}


            down_time_details = [] # [{'start': 1735897800, 'duration': 9000}, {'start': 1735897800, 'duration': 9000}]

            # If an ASN is down, find its start time and the duration
            if down_time_counter > 0:
                down_time_details = []

                # Iterate over data to get start and duration field
                for d in data:
                    start_duration = { }
                    start_duration["start"] = d["start"]
                    start_duration["duration"] = d["duration"]
                    down_time_details.append(start_duration)

        else:
            print(f"Error: Received status code {response.status_code}")
        merge_all.append(down_time_counter_dict)
        merge_all.append(down_time_details)
        down_time_asn[asn] = merge_all

    # Save result into a json file
    with open("../data/"+country+"_asn_ioda_down_detail_nov_2024.json", "w") as outfile:
        json.dump(down_time_asn, outfile)
    print(f"Completed for {country}")